In [ ]:
!git clone https://github.com/llm-fine-tuning/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 12759, done.
remote: Total 12759 (delta 0), reused 0 (delta 0), pack-reused 12759 (from 1)
Receiving objects: 100% (12759/12759), 219.55 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (9390/9390), done.
Updating files: 100% (220/220), done.


In [ ]:
%cd LLaMA-Factory
%pwd

/workspace/LLaMA-Factory


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/workspace/LLaMA-Factory'

In [ ]:
!pip install -r requirements.txt deepspeed==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 36.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.7/163.7 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 20.6 MB/s eta 0:00:00
INFO: pip is looking 

`train_sft.sh` 열어서 deepspeed 코드 수정
```script
deepspeed --num_gpus 2 --master_port=9901 src/train.py \
	--deepspeed ds_z3_config.json \
    --stage sft \
    --do_train \
    --model_name_or_path allganize/Llama-3-Alpha-Ko-8B-Instruct \
    --dataset text_to_sql_data \
    --template llama3-text-to-sql \
    --finetuning_type lora \
    --lora_target all \
    --output_dir checkpoint \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 1e-4 \
    --num_train_epochs 10.0 \
    --report_to none \
    --bf16
```

학습 수행(터미널에서 진행)
```bash
ls -al
chmod 777 train_sft.sh
nohup ./train_sft.sh > train.log &
```

In [ ]:
# merge 수행하기
!python merge.py \
--base_model_name_or_path allganize/Llama-3-Alpha-Ko-8B-Instruct \
--peft_model_path ./checkpoint/checkpoint-200 \
--output_dir ./output_dir

Loading base model: allganize/Llama-3-Alpha-Ko-8B-Instruct
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:16<00:00,  4.24s/it]
Loading PEFT: ./checkpoint/checkpoint-200
Running merge_and_unload
[2024-10-16 05:10:12,117] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Model saved to ./output_dir


# 테스트

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
input_text = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 `입력 텍스트:`와 `DDL Statements`를 참고하여 SQL 쿼리를 작성하는 SQL 쿼리 작성기입니다.
`쿼리 작성:`이라고 하면 바로 SQL 쿼리를 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력 텍스트: 각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>쿼리 작성:"""

In [ ]:
# 기존 모델
tokenizer = AutoTokenizer.from_pretrained("allganize/Llama-3-Alpha-Ko-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("allganize/Llama-3-Alpha-Ko-8B-Instruct")

model = torch.nn.DataParallel(model).cuda()

inputs = tokenizer(input_text, return_tensors='pt')
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 `입력 텍스트:`와 `DDL Statements`를 참고하여 SQL 쿼리를 작성하는 SQL 쿼리 작성기입니다.
`쿼리 작성:`이라고 하면 바로 SQL 쿼리를 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력 텍스트: 각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>쿼리 작성: 각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?<|eot_id|>


In [ ]:
# 학습된 모델
tokenizer = AutoTokenizer.from_pretrained("./output_dir")
model = AutoModelForCausalLM.from_pretrained("./output_dir")

model = torch.nn.DataParallel(model).cuda()

inputs = tokenizer(input_text, return_tensors='pt')
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

당신은 주어진 `입력 텍스트:`와 `DDL Statements`를 참고하여 SQL 쿼리를 작성하는 SQL 쿼리 작성기입니다.
`쿼리 작성:`이라고 하면 바로 SQL 쿼리를 작성하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

입력 텍스트: 각 영업사원이 판매한 목재의 총량은 무엇이며, 영업사원별로 정렬된 목록은 무엇인가요?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');

위의 테이블 명세와 사용자의 입력 텍스트를 바탕으로 SQL 쿼리를 작성합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>쿼리 작성: SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id 

# 허깅페이스 업로드

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
username="YOUR_HUGGINGFACE_USERNAME"

In [ ]:
MODEL_NAME = "YOUR_MODEL_NAME"

In [ ]:
api.create_repo(
    token="YOUR_HUGGINGFACE_ACCESS_TOKEN",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="YOUR_HUGGINGFACE_ACCESS_TOKEN",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir"
)

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/minoso/t2s-llama-3/commit/27c28bc3766edcd84929da1a09cc0cca7d8ef683', commit_message='Upload folder using huggingface_hub', commit_description='', oid='27c28bc3766edcd84929da1a09cc0cca7d8ef683', pr_url=None, repo_url=RepoUrl('https://huggingface.co/minoso/t2s-llama-3', endpoint='https://huggingface.co', repo_type='model', repo_id='minoso/t2s-llama-3'), pr_revision=None, pr_num=None)

In [ ]:
# 학습된 모델
tokenizer = AutoTokenizer.from_pretrained(f"{username}/{MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}")

model = torch.nn.DataParallel(model).cuda()

inputs = tokenizer(input_text, return_tensors='pt')
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.module.generate(input_ids=inputs["input_ids"].to('cuda'), max_new_tokens=512, eos_token_id=eos_token_id)
    print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

KeyboardInterrupt: 